# RNN Model

In this section we will implement the RNN Model Recurrent Neural Network. 

The RNN (Recurrent Neural Network) Forecaster is a vanilla RNN model designed for multivariate time series forecasting. It processes sequential data using recurrent connections that maintain a hidden state across time steps.

Layer Breakdown:

- **RNN Layers:** 2 stacked RNN layers with Tanh activation
- **Hidden Size:** 64 units per layer
- **Dropout:** Applied between RNN layers (if >1 layer) and before final output
- **Output Layer:** Single fully connected layer producing 1-step forecast

**Advantages**
- **Simplicity:** Simpler architecture than LSTM/GRU (only hidden state, no cell state)
- **Speed:** Faster training due to fewer parameters
- **Sequential Processing:** Captures temporal dependencies in time series data

**Limitations**

- **Vanishing Gradients:** May struggle with long-term dependencies (sequences >10-20 steps)
- **Limited Memory:** No gating mechanisms to control information flow like LSTM/GRU


In [1]:
import torch
import torch.nn as nn

## Model

In [2]:
class RNNForecaster(nn.Module):
    """
    Vanilla RNN model for MULTIVARIATE time series forecasting.
    Architecture: RNN -> Dropout -> RNN -> Dropout -> Fully Connected
    Takes multiple input features at each timestep.
    
    Simpler than LSTM - no cell state, only hidden state.
    Faster training but may struggle with long-term dependencies.
    """
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2):
        """
        Args:
            input_size: Number of input features (Value + year + month + one-hot)
            hidden_size: RNN hidden dimension
            num_layers: Number of RNN layers
            dropout: Dropout rate
        """
        super(RNNForecaster, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        
        # RNN layers (using Tanh activation by default)
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            nonlinearity='tanh'  # Can also use 'relu'
        )
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # x shape: (batch_size, seq_length, input_size)
        
        # RNN forward pass
        # rnn_out: (batch_size, seq_length, hidden_size)
        # h_n: (num_layers, batch_size, hidden_size)
        rnn_out, h_n = self.rnn(x)
        
        # Take the output from the last time step
        last_output = rnn_out[:, -1, :]  # Shape: (batch_size, hidden_size)
        
        # Apply dropout
        out = self.dropout(last_output)
        
        # Fully connected layer
        out = self.fc(out)  # Shape: (batch_size, 1)
        
        return out

In [ ]:
!cd ..&& python ./neuralts/main.py --model RNN --epochs 25 --batch-size 64 --learning-rate 0.0005

✓ MPS device is available
Device: mps
DATA OVERVIEW
Original data shape: (107922, 29)
Unique time series: 1502
Date range: 2018-01-31 00:00:00 to 2025-09-30 00:00:00
Total observations: 107,922

Original DataFrame:
  NaN values: 0
  Inf values: 0
  Value range: [0.00, 22335.00]

Zero values in 'Value' column: 20818
Values < 0.01: 20818

FOLD CONFIGURATION

Fold 1:
  Training data: up to 2024-09-30
  Test period: 2024-10-01 to 2024-12-31

Fold 2:
  Training data: up to 2024-12-31
  Test period: 2025-01-01 to 2025-03-31

Fold 3:
  Training data: up to 2025-06-30
  Test period: 2025-07-01 to 2025-09-30

FOLD 1: 2024-10-01 to 2024-12-31

Filtered training data up to 2024-09-30
  Training observations: 90,494
  Date range: 2018-01-31 00:00:00 to 2024-09-30 00:00:00

------------------------------------------------------------
STEP 1: Creating datasets for model development
------------------------------------------------------------
Creating dataset with 1478 time series...
Additional featu